# HIGGS example using IBM PowerAI Snap ML

In this example we will train a Logistic Regression model on the HIGGS dataset, using both scikit-learn and snap-ml-local.

The HIGGS dataset is avaliable in the UCI machine learning repository.

To avoid 'kernel restart' problem increase CPU and memory for the jupyter environment (e.g. memory 10GB, CPU 100) and restart it

### Download input data

Two wget commands are given below for downloading input HIGGS dataset - one for reduced dataset and another for bigger/full dataset. Many times better perfomance of snapML training is seen with bigger dataset.

You can comment the line containing the wget command to avoid downloading dataset again if running the same wget command more than once. Similarly preprocessing code can be commented out second time onwards if using the same dataset.

In [1]:
# About 3 times better training time with snapML compared to sklearn with this full HIGGS dataset
!mkdir -p data; cd data; wget -nc https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/HIGGS.bz2; bunzip2 HIGGS.bz2; cd ../

# Download reduced dataset
#!mkdir -p data; cd data; wget -O HIGGS -nc https://ibm.box.com/shared/static/v684mqemrrz9o9gsko4ox5l30t6ncqag; cd ../

File ‘HIGGS.bz2’ already there; not retrieving.

bunzip2: Output file HIGGS already exists.


### Preprocess the data

In [2]:
# Preprocess the data
from sklearn.datasets import load_svmlight_file

defaultPath = "."

X,y = load_svmlight_file(defaultPath + "/data/HIGGS")

# Make the train-test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=42)

# Convert to numpy ararys
import numpy as np
X_train = np.array(X_train.todense())
X_test  = np.array(X_test.todense())

# Normalize the training data
from sklearn.preprocessing import normalize
X_train = normalize(X_train, axis=1, norm='l1')
X_test  = normalize(X_test,  axis=1, norm='l1')

# Save the dense matrices
np.save(defaultPath + "/data/HIGGS.X_train", X_train)
np.save(defaultPath + "/data/HIGGS.X_test",  X_test)

# Save the labels
np.save(defaultPath + "/data/HIGGS.y_train", y_train)
np.save(defaultPath + "/data/HIGGS.y_test", y_test)

### Training and Evaluating a Logistic Regression Model using GPU

In [3]:
# Training and Evaluating a Logistic Regression Model using GPU
from scipy import sparse

# Load the data
import time
import numpy as np
import argparse

defaultPath = "."

t0 = time.time()
X_train = np.load(defaultPath + "/data/HIGGS.X_train.npy")
X_test  = np.load(defaultPath + "/data/HIGGS.X_test.npy")
y_train = np.load(defaultPath + "/data/HIGGS.y_train.npy")
y_test  = np.load(defaultPath + "/data/HIGGS.y_test.npy")
print("Data load time (s):  {0:.2f}".format(time.time()-t0))

# Import the LogisticRegression from snap.ml
from snap_ml import LogisticRegression
lr = LogisticRegression(use_gpu=True, max_iter=15, dual=True, num_threads=32, device_ids=[])

# Training
t0 = time.time()
lr.fit(X_train, y_train)
print("[snap.ml] Training time (s):  {0:.2f}".format(time.time()-t0))

# Inference
proba_test = lr.predict_proba(X_test)

# Evaluate log-loss on test set
from sklearn.metrics import log_loss
logloss_snap = log_loss(y_test, proba_test)
print("[snap.ml] Logarithmic loss:   {0:.4f}".format(logloss_snap))

# Import the LogisticRegression from sklearn
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(fit_intercept=False, dual=True)

# Training time
t0 = time.time()
lr.fit(X_train, y_train)
print("[sklearn] Training time (s):  {0:.2f}".format(time.time()-t0))

# Inference
proba_test = lr.predict_proba(X_test)

# Evaluate log-loss on test set
logloss_sklearn = log_loss(y_test, proba_test)
print("[sklearn] Logarithmic loss:   {0:.4f}".format(logloss_sklearn))

Data load time (s):  0.26
[Warning] Parameter device_ids not set. The training will run on GPU 0.
[snap.ml] Training time (s):  2.47
[snap.ml] Logarithmic loss:   0.6374
[sklearn] Training time (s):  74.43
[sklearn] Logarithmic loss:   0.6374


### Training and Evaluating a Logistic Regression Model using CPU

In [4]:
# Training and Evaluating a Logistic Regression Model using CPU
from scipy import sparse

# Load the data
import time
import numpy as np
import argparse

defaultPath = "."

t0 = time.time()
X_train = np.load(defaultPath + "/data/HIGGS.X_train.npy")
X_test  = np.load(defaultPath + "/data/HIGGS.X_test.npy")
y_train = np.load(defaultPath + "/data/HIGGS.y_train.npy")
y_test  = np.load(defaultPath + "/data/HIGGS.y_test.npy")
print("Data load time (s):  {0:.2f}".format(time.time()-t0))

# Import the LogisticRegression from snap.ml
from snap_ml import LogisticRegression
lr = LogisticRegression(use_gpu=False, max_iter=15, dual=True, num_threads=32, device_ids=[])

# Training
t0 = time.time()
lr.fit(X_train, y_train)
print("[snap.ml] Training time (s):  {0:.2f}".format(time.time()-t0))

# Inference
proba_test = lr.predict_proba(X_test)

# Evaluate log-loss on test set
from sklearn.metrics import log_loss
logloss_snap = log_loss(y_test, proba_test)
print("[snap.ml] Logarithmic loss:   {0:.4f}".format(logloss_snap))

# Import the LogisticRegression from sklearn
from sklearn.linear_model import LogisticRegression
lr = LogisticRegression(fit_intercept=False, dual=True)

# Training time
t0 = time.time()
lr.fit(X_train, y_train)
print("[sklearn] Training time (s):  {0:.2f}".format(time.time()-t0))

# Inference
proba_test = lr.predict_proba(X_test)

# Evaluate log-loss on test set
logloss_sklearn = log_loss(y_test, proba_test)
print("[sklearn] Logarithmic loss:   {0:.4f}".format(logloss_sklearn))

Data load time (s):  0.19
[Info][Experimental feature] Training will run in multi-threaded mode on CPU. Training convergence is not guaranteed.
[snap.ml] Training time (s):  23.47
[snap.ml] Logarithmic loss:   0.6379
[sklearn] Training time (s):  69.51
[sklearn] Logarithmic loss:   0.6374


&copy; Copyright IBM Corporation 2018